In [ ]:
# If you don't have the moduel please install them for this notebook
# !pip install Bio

In [1]:
import re
import numpy as np
import pandas as pd
from collections import Counter
import itertools
from Bio.SeqUtils.ProtParam import ProteinAnalysis

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
import random
from sklearn.model_selection import cross_validate
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report
)

import matplotlib.pyplot as plt
import seaborn as sns

# Read in data that was generated from the analysis and preprocessing steps

In [2]:
# Read in data
data = pd.read_csv('metadata_org_w_features.csv')

# Information and sample data

In [3]:
# information on data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681 entries, 0 to 680
Columns: 589 entries, Entry to Gravy
dtypes: float64(584), int64(1), object(4)
memory usage: 3.1+ MB


In [4]:
# describe on data
data.describe()

SequenceLength           A           C           D           E  \
count      681.000000  681.000000  681.000000  681.000000  681.000000   
mean       888.660793    0.071173    0.023348    0.044993    0.055845   
std        827.859592    0.022832    0.012300    0.013231    0.017925   
min        103.000000    0.024283    0.000000    0.007605    0.005714   
25%        384.000000    0.056695    0.015251    0.037879    0.044280   
50%        676.000000    0.067460    0.021417    0.046046    0.058594   
75%       1015.000000    0.081784    0.030753    0.053254    0.067669   
max       7096.000000    0.164000    0.074873    0.116598    0.123102   

                F           G           H           I           K  ...  \
count  681.000000  681.000000  681.000000  681.000000  681.000000  ...   
mean     0.046743    0.065586    0.022028    0.056255    0.047262  ...   
std      0.015308    0.018840    0.007750    0.018204    0.017596  ...   
min      0.007782    0.016807    0.004065    0.009840    0.002915  ...   
25%      0.035479    0.052764    0.016825    0.044171    0.034237  ...   
50%      0.044747    0.065963    0.021544    0.056726    0.048017  ...   
75%      0.055838    0.078212    0.026769    0.068000    0.058943  ...   
max      0.110701    0.153509    0.056093    0.125461    0.104956  ...   

              DDD  MolecularWeight  IsoelectricPoint  Aromaticity  \
count  681.000000       681.000000        681.000000   681.000000   
mean     0.002206     99103.867857          7.097584     0.096863   
std      0.003108     92308.456460          1.551349     0.021673   
min      0.000000     11710.270000          4.383673     0.033074   
25%      0.000000     42771.083500          5.723255     0.082587   
50%      0.001377     74697.663500          6.556855     0.094744   
75%      0.003229    112207.097800          8.660054     0.109974   
max      0.031637    794048.925800         11.114366     0.180812   

       InstabilityIndex  Flexibility       Helix       Sheet        Turn  \
count        681.000000   681.000000  681.000000  681.000000  681.000000   
mean          42.160570     0.995021    0.304995    0.279123    0.389817   
std            7.412366     0.006859    0.029081    0.032611    0.043874   
min           15.359896     0.974438    0.222539    0.183150    0.248630   
25%           37.259593     0.991538    0.284024    0.257261    0.361333   
50%           42.218182     0.996686    0.302326    0.277000    0.382857   
75%           46.958534     0.999503    0.323615    0.300946    0.416667   
max           66.024733     1.013136    0.403005    0.419863    0.546125   

            Gravy  
count  681.000000  
mean    -0.051027  
std      0.325872  
min     -0.757339  
25%     -0.284269  
50%     -0.120350  
75%      0.136484  
max      1.008772  

[8 rows x 585 columns]

In [5]:
data.head()

Entry                                      CleanSequence Selected_PDB  \
0  P21611  MGKAAAVVLVTLVALLGLAQADLTPKVQVYSRFPASAGTKNVLNCF...         3p73   
1  Q66GT5  MAASAWLEAGLARVLFYPTLLYTVFRGRVRGPAHRDWYHRIDHTVL...         3rgo   
2  Q9Y006  MNLTIKEEDFTNTFMKNEESFNTFRVTKVKRWNAKRLFKILFVTVF...         3qvc   
3  P05622  MGLPGVIPALVLRGQLLLSVLWLLGPQTSRGLVITPPGPEFVLNIS...         1aya   
4  P06343  MALQIPSLLLLAAVVVLTVLSSPGTEGGNSERHFVHQFQPFCYFTN...         1d9k   

  ProteinClass  SequenceLength         A         C         D         E  \
0          MHC             119  0.109244  0.016807  0.058824  0.042017   
1  Phosphatase             193  0.108808  0.015544  0.020725  0.067358   
2     Protease             451  0.028825  0.008869  0.048780  0.068736   
3          RTK            1098  0.043716  0.017304  0.056466  0.069217   
4          MHC             263  0.041825  0.019011  0.022814  0.076046   

          F  ...       DDD  MolecularWeight  IsoelectricPoint  Aromaticity  \
0  0.058824  ...  0.000000       13041.7928          5.834204     0.109244   
1  0.025907  ...  0.000000       21942.2575          9.739448     0.082902   
2  0.079823  ...  0.002227       51692.5641          8.043414     0.137472   
3  0.030055  ...  0.003650      122788.2434          4.994010     0.083789   
4  0.038023  ...  0.000000       29966.8989          8.991680     0.091255   

   InstabilityIndex  Flexibility     Helix     Sheet      Turn     Gravy  
0         22.404202     0.996347  0.336134  0.277311  0.369748 -0.047059  
1         37.915078     0.992205  0.367876  0.186528  0.383420 -0.161140  
2         37.618204     1.002078  0.305987  0.290466  0.434590 -0.249667  
3         47.564763     0.999045  0.281421  0.311475  0.385246 -0.203097  
4         46.750989     0.995690  0.262357  0.254753  0.399240 -0.303042  

[5 rows x 589 columns]

# Extract the relevant fields from the dataset

In [6]:
# Entry and ProteinClass
df = data.loc[:,['Entry', 'ProteinClass']]
# PDB
selected_PDB = data.loc[:,['Selected_PDB']]
# Sequence
seq = data.loc[:,['CleanSequence']]
# Sequence Length
seq_L = data.loc[:,['SequenceLength']]
# Amino Acid Frequencies
freq = data.iloc[:, 5 : 5+20]
# All possible dipeptide frequencies
dipep = data.iloc[:, 5+20 : 5+20 + 400]
# Reduced Amino Acid Alphabet Frequencies
red_freq = data.iloc[:, 5+20 + 400 : 5+20 + 400 + 5]
# N-Gram Profiles of Reduced Amino Acid Alphabet
red_ngram = data.iloc[:, 5+20 + 400 + 5 : 5+20 + 400 + 5 + 150]
# Protein Properties
prop = data.iloc[:, 5+20 + 400 + 5 + 150 :]

In [7]:
prop.head()

MolecularWeight  IsoelectricPoint  Aromaticity  InstabilityIndex  \
0       13041.7928          5.834204     0.109244         22.404202   
1       21942.2575          9.739448     0.082902         37.915078   
2       51692.5641          8.043414     0.137472         37.618204   
3      122788.2434          4.994010     0.083789         47.564763   
4       29966.8989          8.991680     0.091255         46.750989   

   Flexibility     Helix     Sheet      Turn     Gravy  
0     0.996347  0.336134  0.277311  0.369748 -0.047059  
1     0.992205  0.367876  0.186528  0.383420 -0.161140  
2     1.002078  0.305987  0.290466  0.434590 -0.249667  
3     0.999045  0.281421  0.311475  0.385246 -0.203097  
4     0.995690  0.262357  0.254753  0.399240 -0.303042

# You can use this section if you wanted to add more fetures to the data

In [8]:
# Feel free to add other feature to this data

# Construct Dataset with any of the extracted relevant fields

In [9]:
# This is an example of construction of the data with freq and red_freq, but you can add and make any field
Dataset = pd.concat([df, freq, red_freq, prop, red_ngram], axis=1)
Dataset.head()

Entry ProteinClass         A         C         D         E         F  \
0  P21611          MHC  0.109244  0.016807  0.058824  0.042017  0.058824   
1  Q66GT5  Phosphatase  0.108808  0.015544  0.020725  0.067358  0.025907   
2  Q9Y006     Protease  0.028825  0.008869  0.048780  0.068736  0.079823   
3  P05622          RTK  0.043716  0.017304  0.056466  0.069217  0.030055   
4  P06343          MHC  0.041825  0.019011  0.022814  0.076046  0.038023   

          G         H         I  ...       DHG       DHF       DHS       DHH  \
0  0.058824  0.025210  0.016807  ...  0.000000  0.000000  0.000000  0.000000   
1  0.051813  0.041451  0.046632  ...  0.000000  0.000000  0.005236  0.000000   
2  0.053215  0.015521  0.062084  ...  0.004454  0.002227  0.002227  0.002227   
3  0.061020  0.021858  0.050091  ...  0.003650  0.000912  0.003650  0.001825   
4  0.072243  0.030418  0.041825  ...  0.003831  0.000000  0.007663  0.003831   

        DHD       DDG       DDF       DDS       DDH       DDD  
0  0.008547  0.000000  0.008547  0.000000  0.000000  0.000000  
1  0.000000  0.005236  0.005236  0.005236  0.000000  0.000000  
2  0.002227  0.000000  0.002227  0.004454  0.002227  0.002227  
3  0.000912  0.004562  0.000912  0.003650  0.002737  0.003650  
4  0.000000  0.000000  0.003831  0.003831  0.000000  0.000000  

[5 rows x 186 columns]

In [10]:
## To Do ##
# Construct your own dataset

# Processing Steps

In [13]:
def processing_data(df):
    # Define X and y
    y = df['ProteinClass']
    X = df.drop(['Entry', 'ProteinClass'], axis=1)
    if 0:
        print("Feature sample:")
        print(X.head())
        print("\nTarget sample:")
        print(y.head())

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

    # Feature scaling (important for logistic regression, gradient boosting, MLP, etc.)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)


    # Defined labels
    labels = np.array(['ATPase', 'Aquaporin', 'Channel', 'GPCR', 'Integrin', 'MHC',
       'Phosphatase', 'Protease', 'RTK', 'Ser:Thr'])
    # Create a Label Encoder
    label_encoder = LabelEncoder()
    # Fit the label encoder on all possible labels, will make the labels [0, 1, 2, 3 ...]
    label_encoder.fit(labels)
    # Transform both training and testing labels
    y_label_encoded = label_encoder.transform(y)
    y_train_label_encoded = label_encoder.transform(y_train)
    y_test_label_encoded = label_encoder.transform(y_test)
    print('first sample label encoded:', y_train_label_encoded[0])
 
    
    

    ### This is where you can start to add more normalization or processing approches if you would like ###

    return (label_encoder, X, X_scaled, X_train, X_train_scaled, X_test, X_test_scaled,
            y, y_label_encoded, y_train, y_train_label_encoded, y_test, y_test_label_encoded)
    

In [14]:
(label_encoder, X, X_scaled, X_train, X_train_scaled, X_test, X_test_scaled, y, y_label_encoded,
y_train, y_train_label_encoded, y_test, y_test_label_encoded)  = processing_data(Dataset)

first sample label encoded: 3


# Creating Model

In [15]:
# random model
class random_model:
    def __init__(self):
        pass
    def fit(self,X, y):
        pass
    def predict(self,X):
        pred = []
        for x in X:
            pred.append(np.random.randint(10))
        return np.array(pred)
    def get_params(self, deep=True):
        return {}
    
# Initialize my random model
my_model = random_model()
# Initialize Logistic Regression model
lr_model = LogisticRegression( C = 1)

## To Do ##
# Create three model here like random forest, leanear model, etc and initialize it
# you can use sklear, torch, etc 

# Train Model

In [17]:
# Fit My Model to the training data
my_model.fit(X_train, y_train)

# Fit the model to the training data
lr_model.fit(X_train_scaled, y_train)

## To Do ##
# Fit your models to the training data you can use
# raw, scaled, or other forms

LogisticRegression(C=1)

# Evaluating model

In [18]:
def evaluate_model(model, X_test, y_test, model_name):

    y_pred = model.predict(X_test)

    # Evaluation
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n=== {model_name} ===")
    print(f"Accuracy for testing data: {accuracy:.4f}")
    return y_pred

In [19]:
# Evaluate my model
y_pred_my_model = evaluate_model(my_model, X_test_scaled, y_test_label_encoded, model_name="My Random Model")
# Convert back to labels
y_pred_my_model = label_encoder.inverse_transform(y_pred_my_model)
# Print my model prediction
# print(y_pred_my_model)

# Evaluate Logistic Regression
y_pred_lr_model = evaluate_model(lr_model, X_test_scaled, y_test, model_name="Logistic Regression")
# Print lr model prediction
# print(y_pred_my_model)


## To Do ##
# Evaluate all three of your models



=== My Random Model ===
Accuracy for testing data: 0.0584

=== Logistic Regression ===
Accuracy for testing data: 0.7372


# 10-fold CV evaluation of a classifier

In [39]:
# 10-fold CV evaluation of a classifier
def eval_clf(_clf, _X, _y):
    model_cv = cross_validate(_clf, _X, _y, cv=10, scoring='accuracy', n_jobs=10, return_estimator=True)
    return model_cv

def evaluate_model_CV(model, X, y, model_name):

    model_cv = eval_clf(model, X, y)
    acc = model_cv["test_score"]

    # Evaluation
    print(f"\n=== {model_name} ===")
    print(f'CV accuracy: {np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')
    return model_cv["estimator"]

In [40]:
# 10-Fold CV of my model
my_model_cv= evaluate_model_CV(random_model(), X, y_label_encoded, "My Random Model CV")

# 10-Fold CV of Logistic Regression
LR_model_cv = evaluate_model_CV(LogisticRegression(), X_scaled, y, "Logistic Regression CV")

## To Do ##
# Perform CV for all your three models


=== My Random Model CV ===
CV accuracy: nan ±nan

=== Logistic Regression CV ===
CV accuracy: 0.74 ±0.024


# Creating Ensemble Model

In [41]:
N_ENSEMBLE = 100

# Initialize my ensuble model
clfs_my_model = [random_model() for _ in range(N_ENSEMBLE)]
clfs_lr_model = [LogisticRegression() for _ in range(N_ENSEMBLE)]

## To Do ##
# Create one Ensemble model

# Ensemble Helper Functions

In [42]:
# Wrapper to use the classifiers with cross_validate and Ensemble
class Eclf(BaseEstimator):
    def __init__(self, ensemble=list(), subsample_ratio=0.1):
        self.ensemble, self.sr = ensemble, subsample_ratio
    
    # Ensemble fit 
    def fit(self, _X, _y, **kwargs):
        self.unique_classes = len(np.unique(_y))
        self.ensemble, self.sr = kwargs['ensemble'], kwargs['subsample_ratio']
        N = _X.shape[0]
        for c in self.ensemble:
            bCheck = True # Check if we got two classes - especially problem for very small subsample
            while bCheck:
                ix = random.sample(range(N), int(self.sr*N))
                if len(np.unique(_y[ix])) == self.unique_classes:
                    bCheck = False
            c.fit(_X[ix,:], _y[ix])

    # Voting for the Predictions
    def predict(self, _X):
        N = _X.shape[0]
        ypred_scores = np.zeros((N, self.unique_classes))
        for c in self.ensemble:
            pred = c.predict_proba(_X)
            for i in range(N):
                ypred_scores[i] += pred[i]
        ypred = np.argmax(ypred_scores, axis=1)
        return ypred
    
    def get_params(self, deep=True):
        return {'ensemble':self.ensemble, 'subsample_ratio':self.sr}
    
# 10-fold CV evaluation of a ensemble classifier
def eval_ensemble_clf(_ensemble_list, _X, _y, _subsample_ratio = 0.1):
    my_cv_model_ensemble = cross_validate(Eclf(), _X, _y, cv=10, scoring='accuracy',
        fit_params={'ensemble':_ensemble_list, 'subsample_ratio':_subsample_ratio},
        n_jobs=10, return_estimator=True)
    return my_cv_model_ensemble

# Ensemble Traning and Evaluation 

In [43]:
# Evaluate ensemble
def evaluate_model_CV_ensemble(model_ensemble, X, y, model_name, subsample_ratio):

    model_cv_ensemble = eval_ensemble_clf(model_ensemble, X, y, _subsample_ratio=subsample_ratio)
    acc = model_cv_ensemble["test_score"]

    # Evaluation
    print(f"\n=== {model_name} ===")
    print(f'Ensemble CV accuracy: {np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')
    return model_cv_ensemble["estimator"]

In [44]:
# Ensemble traning and evaluation 
clfs_lr_model_ensemble = evaluate_model_CV_ensemble(clfs_lr_model, X_scaled, y_label_encoded,'Logistic Regression CV Ensemble', subsample_ratio=0.6)


## To Do ##
# Train and Evaluate one of your Ensemble models

c:\Users\yasinas1\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(



=== Logistic Regression CV Ensemble ===
Ensemble CV accuracy: 0.77 ±0.026


In [27]:
# This code is provided if you want more control of your Cross Validation
def ensemble_fit(_clfs, _X, _y, subsample_ratio, unique_classes):
    N = _X.shape[0]
    for c in _clfs:
        bCheck = True # Check if we got two classes - especially problem for very small subsample
        while bCheck:
            ix = random.sample(range(N), int(subsample_ratio*N))
            if len(np.unique(_y[ix])) ==unique_classes:
                bCheck = False
        c.fit(_X[ix,:], _y[ix])

def ensemble_predict(_clfs, _X, unique_classes):
    N = _X.shape[0]
    ypred_scores = np.zeros((N, unique_classes))
    for c in _clfs:
        pred = c.predict_proba(_X)
        for i in range(N):
            ypred_scores[i] += pred[i]
        ypred = np.argmax(ypred_scores, axis=1)
    return ypred

# 10-fold CV evaluation of a classifier, using an ensemble
def eval_ensemble_clf_trivial(_ensemble_list, _X, _y, _subsample_ratio):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import accuracy_score
    acc = []
    kf = StratifiedKFold(n_splits=10, shuffle=False)
    unique_classes = len(np.unique(_y))
    for tr_i, ts_i in kf.split(_X, _y):
        ensemble_fit(_ensemble_list, _X[tr_i], _y[tr_i], _subsample_ratio, unique_classes)
        y_pred = ensemble_predict(_ensemble_list, _X[ts_i], unique_classes)
        acc += [accuracy_score(_y[ts_i], y_pred)]
    return acc # return list

In [ ]:
# Train and Evaluate Ensemble model for the travial 
acc = eval_ensemble_clf_trivial(clfs_lr_model, X_scaled, y_label_encoded, 0.1)
print(f'Ensemble LR accuracy: {np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

Ensemble LR accuracy= 0.70 ±0.030


# Save Format

In [ ]:
def save_predictions(_fn, _y_pred, _df):
    import csv
    with open(_fn, 'w') as fout:
        writer = csv.writer(fout, delimiter=',', lineterminator='\n')
        writer.writerow(['Entry', 'ProteinClass'])
        for y, Entry in zip(_df['Entry'], _y_pred):
            writer.writerow([y, Entry])